# Import libraries

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import cv2
import tensorflow as tf
import random 

from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Explore the images from image preparation

In [29]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
# img_loc = 'C:/Users/Asus/Documents/Thesis/Me/Dataset'
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
img_loc = "gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered"
print(img_loc)

gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered


In [31]:
data_dir = pathlib.Path(img_loc)

In [32]:
data_dir

PosixPath('gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered')

In [33]:
# img_eu_loc = img_loc + '/european-street-fashion-output'
# img_jp_loc = img_loc + '/harajuku-kawaii-street-fashion-output'
# img_th_loc = img_loc + '/thai-street-fashion-output'

img_jp_loc = img_loc + '/ha-output/crop'
img_th_loc = img_loc + '/th-output/crop'

In [34]:
img_jp_loc

'gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop'

In [35]:
dim_i_list = []
dim_j_list = []
img_list = []
label_list = []
idx = 0

#### THAI

In [36]:
for img_name in list(os.listdir(img_th_loc)):
    img_loc = '{}/{}'.format(img_th_loc, img_name)
    img = cv2.imread(img_loc)
    print('{}: {} - {}'.format(img_loc, img.shape, idx))
    dim_i_list.append(img.shape[0])
    dim_j_list.append(img.shape[1])
    img_list.append(img)
    label_list.append(0)
    idx = idx+1

gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00000.png: (854, 933, 3) - 0
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00002.png: (470, 248, 3) - 1
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00001.png: (465, 209, 3) - 2
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00007.png: (244, 238, 3) - 3
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00008.png: (266, 245, 3) - 4
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00004.png: (321, 411, 3) - 5
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00005.png: (376, 369, 3) - 6
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/crop/test00009.png: (447, 251, 3) - 7
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/th-output/cro

#### KAWAII

In [37]:
for img_name in list(os.listdir(img_jp_loc)):
    img_loc = '{}/{}'.format(img_jp_loc, img_name)
    img = cv2.imread(img_loc)
    print('{}: {} - {}'.format(img_loc, img.shape, idx))
    dim_i_list.append(img.shape[0])
    dim_j_list.append(img.shape[1])
    img_list.append(img)
    label_list.append(1)
    idx = idx+1

gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00000.png: (234, 120, 3) - 186
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00001.png: (388, 267, 3) - 187
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00004.png: (582, 238, 3) - 188
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00007.png: (476, 399, 3) - 189
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00002.png: (622, 278, 3) - 190
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00008.png: (498, 309, 3) - 191
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00012.png: (313, 252, 3) - 192
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filtered/ha-output/crop/test00011.png: (191, 277, 3) - 193
gdrive/MyDrive/NewThesis_Poonyawat_Woottisart/val_dataset_filter

In [38]:
np.mean(dim_i_list), np.std(dim_i_list)

(518.472602739726, 190.7206482994075)

In [39]:
np.mean(dim_j_list), np.std(dim_j_list)

(303.78767123287673, 113.9774496384425)

In [40]:
len(dim_i_list)

292

In [41]:
# 286 - japanese | 488 - thai

# Image Scaling

In [42]:
h = 400
w = 160

img_scaling = [np.zeros(shape=(h, w, 3)) for i in range(len(img_list))]
for i in range(len(img_list)):
    img_scaling[i] = cv2.resize(img_list[i], dsize=(w, h), interpolation = cv2.INTER_CUBIC)

In [43]:
len(img_scaling), len(label_list)

(292, 292)

In [44]:
sample_test_jp = img_scaling[203]
sample_test_th = img_scaling[92]

In [45]:
len(img_scaling), len(img_scaling[0]), len(img_scaling[0][0]), len(img_scaling[0][0][0])

(292, 400, 160, 3)

# Functions

In [46]:
# network creating from given parameter
# input: 
#       number of layers in network (integer), 
#       list of infomation for each layer (list of dictionaries), 
#       list of number of nodes for each dense layer (list of integer(s)), 
#       (optional) list of pooling type for each layer (list of string)
# output: 
#       network topology

def gen_network(n_layers, layers_list, dense_layer_node_list, pooling_set=None): 
    
    # layer_list = list of layers those combined with n_node, padding, activation_fn
    
    return_network = [layers.Rescaling(1./255, input_shape=(h, w, 3))]
    
    # Convolutional Neural Network
    
    for i in range(n_layers):
        temp_conv = layers.Conv2D(layers_list[i]['n_node'], 3, padding=layers_list[i]['padding'], activation=layers_list[i]['activ_fn'])
        return_network.append(temp_conv)
        if(pooling_set==None):
            temp_pooling = layers.MaxPooling2D()
        return_network.append(temp_pooling)
    
    # Dense Layer
    
    return_network.append(layers.Flatten())
    
    for n_node_dense in dense_layer_node_list:
        temp_dense = layers.Dense(n_node_dense, activation='relu')
        return_network.append(temp_dense)
        
    return_network.append(layers.Dense(2))
    
    return return_network

In [47]:
# get model and model parameter from checkpoint
# input: filename (string)
# output: set of model parameters, model

def get_prev_model(filename):

    f = open('gdrive/MyDrive/Colab Notebooks/checkpoints_crop/{}.txt'.format(filename), 'r')

    paramList = []

    for line in f:
        tmp = line.replace('\\n', '')
        paramList.append(line)

    f.close()

    tmp = paramList[1].strip('][').split(', ')
    paramList[1] = [None for i in range(len(tmp))]
    for i in range(len(tmp)):
        if(i%3==0):
            paramList[1][i] = tmp[i].split(': ')[1].replace('}', '')
        else:
            paramList[1][i] = tmp[i].split(': ')[1].replace('}', '').split('\'')[1]
    paramList[1][-1] = paramList[1][-1].split(']')[0]

    tmp = []
    idx = 0
    while(idx<len(paramList[1])):
        tmp.append({'n_node': int(paramList[1][idx]), 'padding': paramList[1][idx+1], 'activ_fn': paramList[1][idx+2]})
        idx = idx+3
    paramList[1] = tmp.copy()

    tmp = paramList[2].strip('][').split(', ')
    paramList[2] = []
    for i in range(len(tmp)-1):
        paramList[2].append(int(tmp[0]))
    paramList[2].append(int(tmp[-1].split(']')[0]))

    param_prev = np.array([int(paramList[0]), paramList[1], paramList[2]])

    net = gen_network(n_layers=int(paramList[0]), layers_list=paramList[1], dense_layer_node_list=paramList[2])
    model_prev = Sequential(net)
    model_prev.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['binary_accuracy'])
    model_prev.load_weights('gdrive/MyDrive/Colab Notebooks/checkpoints_crop/{}'.format(filename))

    return param_prev, model_prev

# Run

In [48]:
best_param, best_model = get_prev_model('best_model')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Final Evaluation

In [49]:
# model evaluation process 
# input: model, list of testing images (list of image arrays), list of labels (list of integers (classes))
# output: -

def evaluate_final(model, x_test, y_test):
    # model.evaluate(x_test, y_test, verbose=2)

    probability_model = tf.keras.Sequential([
                                             model,
                                             tf.keras.layers.Softmax()])
    x_test = np.array(x_test, ndmin=4)
    pred_prob = probability_model(x_test).numpy()
    
    pred_class = []
    for i in range(len(pred_prob)):
        pred_class.append(np.where(pred_prob[i][0]>pred_prob[i][1], 0, 1))
        print('Actual: {} | Predicted: {} | Prob class 1: {}'.format(y_test[i], pred_class[i], pred_prob[i][1]))
    print(confusion_matrix(y_test, pred_class))
    acc_score = accuracy_score(y_test, pred_class)
    prec_score = precision_score(y_test, pred_class)
    rec_score = recall_score(y_test, pred_class)
    print('accuracy_score: {}'.format(acc_score))
    print('precision_score: {}'.format(prec_score))
    print('recall_score: {}'.format(rec_score))

    if((prec_score==0)&(rec_score==0)):
        f1_score = 0
    else:
        f1_score = 2*prec_score*rec_score/(prec_score+rec_score)


In [50]:
best_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 400, 160, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 400, 160, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 200, 80, 16)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 200, 80, 20)       2900      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 100, 40, 20)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 100, 40, 8)       

In [51]:
evaluate_final(best_model, img_scaling, label_list)

Actual: 0 | Predicted: 0 | Prob class 1: 0.22360020875930786
Actual: 0 | Predicted: 0 | Prob class 1: 0.04416368901729584
Actual: 0 | Predicted: 0 | Prob class 1: 0.24403701722621918
Actual: 0 | Predicted: 1 | Prob class 1: 0.8884128928184509
Actual: 0 | Predicted: 0 | Prob class 1: 0.21783451735973358
Actual: 0 | Predicted: 1 | Prob class 1: 0.6560224294662476
Actual: 0 | Predicted: 0 | Prob class 1: 0.18902571499347687
Actual: 0 | Predicted: 0 | Prob class 1: 0.0016083132941275835
Actual: 0 | Predicted: 0 | Prob class 1: 0.12225527316331863
Actual: 0 | Predicted: 0 | Prob class 1: 0.06027277559041977
Actual: 0 | Predicted: 0 | Prob class 1: 0.3061928451061249
Actual: 0 | Predicted: 0 | Prob class 1: 0.08679144829511642
Actual: 0 | Predicted: 0 | Prob class 1: 0.39950743317604065
Actual: 0 | Predicted: 0 | Prob class 1: 0.0006467806524597108
Actual: 0 | Predicted: 0 | Prob class 1: 0.0690225288271904
Actual: 0 | Predicted: 0 | Prob class 1: 0.07624117285013199
Actual: 0 | Predicted: 0

In [52]:
probability_model = tf.keras.Sequential([
                                         best_model,
                                         tf.keras.layers.Softmax()])

# probability_model = tf.keras.Sequential([
#                                          model_prev,
#                                          tf.keras.layers.Softmax()])
prediction_jp = probability_model(sample_test_jp.reshape(1, h, w, 3)).numpy()
prediction_th = probability_model(sample_test_th.reshape(1, h, w, 3)).numpy()

In [53]:
prediction_jp

array([[0.00282872, 0.99717134]], dtype=float32)

In [54]:
prediction_th

array([[9.9999213e-01, 7.8705061e-06]], dtype=float32)